In [1]:
%load_ext autotime

In [2]:
import json
import time
from datetime import datetime

from etltools import s3
from test_client import LambdaCalculatorTestClient

In [3]:
client = LambdaCalculatorTestClient()

# Test Cost Breakdown

In [4]:
uid = '1000101'
pids = ['2820028008119', '2820088001036']

client.calculate_breakdown(uid, pids)

u'[{"picwell_id": "2820028008119", "uid": "1000101", "covered_breakdown": {"composite": 2730.6020000000003, "in_network": 2730.6020000000003, "categories": {"25": 350.0, "13": 106.19, "12": 721.0799999999999, "15": 179.73000000000002, "17": 680.0, "49": 12.794, "19": 80.0, "32": 0.0, "16": 0.0, "41": 0.0, "0": 0.0, "7": 520.8080000000001, "30": 80.0}, "out_network": 0.0}, "uncovered": 0.0, "deductible_breakdown": {"composite": 0.0, "in_network": 0.0, "categories": {"25": 0.0, "13": 0.0, "12": 0.0, "15": 0.0, "17": 0.0, "49": 0.0, "19": 0.0, "32": 0.0, "16": 0.0, "41": 0.0, "0": 0.0, "7": 0.0, "30": 0.0}, "out_network": 0.0}, "oop": 2730.6020000000003, "allowed": 14459.270000000004, "uncovered_breakdown": {"composite": 0.0, "in_network": 0.0, "categories": {"25": 0.0, "13": 0.0, "12": 0.0, "15": 0.0, "17": 0.0, "49": 0.0, "19": 0.0, "32": 0.0, "16": 0.0, "41": 0.0, "0": 0.0, "7": 0.0, "30": 0.0}, "out_network": 0.0}}, {"picwell_id": "2820088001036", "uid": "1000101", "covered_breakdown"

time: 595 ms


# Test Batch Calculation

In [4]:
uids = s3.read_json('s3n://picwell.sandbox.medicare/samples/philadelphia-2015')
requests_per_second = 100
print '{} uids read'.format(len(uids))

1352473 uids read


In [5]:
uids[:10]

[u'31970436701',
 u'1674435601',
 u'204451602',
 u'3158762201',
 u'595989402',
 u'894536502',
 u'30498125901',
 u'3173810801',
 u'1789308302',
 u'986282601']

In [32]:


for uid in uids[:100]:
#     client.calculate_async(uid, months=['01'])
    start_time = datetime.now()
    client.calculate_async(uid)
    print (datetime.now() - start_time).total_seconds()
    
    #time.sleep(1.0/requests_per_second)  

0.084332
0.158705
0.088289
0.186873
0.087814
0.152434
0.087913
0.099402
0.096584
0.129873
0.079152
0.061407
0.096875
0.104571
0.110631
0.133801
0.10081
0.076163
0.093521
0.135543
0.082093
0.126071
0.081005
0.086527
0.085486
0.087119
0.087784
0.093036
0.150434
0.140938
0.091262
0.110004
0.084024
0.071304
0.102804
0.209344
0.114169
0.148513
0.260177
0.100243
0.084655
0.086553
0.146651
0.119328
0.078969
0.084882
0.095827
0.259608
0.082847
0.088391
0.098038
0.088544
0.095789
0.096906
0.078592
0.090654
0.086977
0.084791
0.090596
0.096579
0.088741
0.089045
0.102221
0.115374
0.121692
0.087564
0.086235
0.090359
0.281282
0.128876
0.086773
0.131909
0.087728
0.078539
0.068069
0.09267
0.086539
0.153036
0.081993
0.076797
0.123167
0.182001
0.092552
0.094504
0.088238
0.09401
0.09419
0.090655
0.0851
0.087434
0.129234
0.088283
0.078164
0.091311
0.088593
0.085626
0.162438
0.11868
0.19269
0.167111


In [8]:
import schedule
%config IPCompleter.greedy=True

fewer_uids = uids[:1000]

1352473
1000
after pop
1352473
999


In [10]:
fewer_uids = uids[:1000]

def call_lambda_with_next_id():
    if len(fewer_uids) > 0:
        client.calculate_async(fewer_uids.pop())

schedule.every(.01).seconds.do(call_lambda_with_next_id)
#schedule.every().hour.do(job)
#schedule.every().day.at("10:30").do(job)
#schedule.every(5).to(10).minutes.do(job)
#schedule.every().monday.do(job)
#schedule.every().wednesday.at("13:15").do(job)

#while True:
#    schedule.run_pending()
#    time.sleep(1)
print('Done')

Done


In [12]:
import sched

In [26]:
scheduler = sched.scheduler(time.time, time.sleep)

def print_event(name):
    print 'EVENT:', time.time(), name

print 'START:', time.time()

start_time = time.time()

increment = .01

print 'plus 100 ms:', time.time() + .01

scheduler.enter(1, 1, print_event, ('first',))
scheduler.enter(5, 1, print_event, ('second',))

count = 1
start_time_for_task = increment

for uid in uids[:1000]:
    start_time_for_task = start_time_for_task + increment
    scheduler.enter(start_time_for_task, 1, print_event, ('call {}'.format(count),))
    count = count + 1


scheduler.run()

START: 1513179368.6
plus 100 ms: 1513179368.61
EVENT: 1513179368.63 call 1
EVENT: 1513179368.64 call 2
EVENT: 1513179368.65 call 3
EVENT: 1513179368.66 call 4
EVENT: 1513179368.67 call 5
EVENT: 1513179368.68 call 6
EVENT: 1513179368.69 call 7
EVENT: 1513179368.7 call 8
EVENT: 1513179368.71 call 9
EVENT: 1513179368.72 call 10
EVENT: 1513179368.73 call 11
EVENT: 1513179368.74 call 12
EVENT: 1513179368.75 call 13
EVENT: 1513179368.76 call 14
EVENT: 1513179368.77 call 15
EVENT: 1513179368.78 call 16
EVENT: 1513179368.79 call 17
EVENT: 1513179368.8 call 18
EVENT: 1513179368.81 call 19
EVENT: 1513179368.82 call 20
EVENT: 1513179368.83 call 21
EVENT: 1513179368.84 call 22
EVENT: 1513179368.85 call 23
EVENT: 1513179368.86 call 24
EVENT: 1513179368.87 call 25
EVENT: 1513179368.88 call 26
EVENT: 1513179368.89 call 27
EVENT: 1513179368.9 call 28
EVENT: 1513179368.91 call 29
EVENT: 1513179368.92 call 30
EVENT: 1513179368.93 call 31
EVENT: 1513179368.94 call 32
EVENT: 1513179368.95 call 33
EVENT: 1

EVENT: 1513179371.52 call 290
EVENT: 1513179371.53 call 291
EVENT: 1513179371.54 call 292
EVENT: 1513179371.55 call 293
EVENT: 1513179371.56 call 294
EVENT: 1513179371.57 call 295
EVENT: 1513179371.58 call 296
EVENT: 1513179371.59 call 297
EVENT: 1513179371.6 call 298
EVENT: 1513179371.61 call 299
EVENT: 1513179371.62 call 300
EVENT: 1513179371.63 call 301
EVENT: 1513179371.64 call 302
EVENT: 1513179371.65 call 303
EVENT: 1513179371.66 call 304
EVENT: 1513179371.67 call 305
EVENT: 1513179371.68 call 306
EVENT: 1513179371.69 call 307
EVENT: 1513179371.7 call 308
EVENT: 1513179371.71 call 309
EVENT: 1513179371.72 call 310
EVENT: 1513179371.73 call 311
EVENT: 1513179371.74 call 312
EVENT: 1513179371.75 call 313
EVENT: 1513179371.76 call 314
EVENT: 1513179371.77 call 315
EVENT: 1513179371.78 call 316
EVENT: 1513179371.79 call 317
EVENT: 1513179371.8 call 318
EVENT: 1513179371.81 call 319
EVENT: 1513179371.82 call 320
EVENT: 1513179371.83 call 321
EVENT: 1513179371.84 call 322
EVENT: 151317

EVENT: 1513179374.41 call 577
EVENT: 1513179374.42 call 578
EVENT: 1513179374.43 call 579
EVENT: 1513179374.44 call 580
EVENT: 1513179374.45 call 581
EVENT: 1513179374.46 call 582
EVENT: 1513179374.47 call 583
EVENT: 1513179374.48 call 584
EVENT: 1513179374.49 call 585
EVENT: 1513179374.5 call 586
EVENT: 1513179374.51 call 587
EVENT: 1513179374.52 call 588
EVENT: 1513179374.53 call 589
EVENT: 1513179374.54 call 590
EVENT: 1513179374.55 call 591
EVENT: 1513179374.56 call 592
EVENT: 1513179374.57 call 593
EVENT: 1513179374.58 call 594
EVENT: 1513179374.59 call 595
EVENT: 1513179374.6 call 596
EVENT: 1513179374.61 call 597
EVENT: 1513179374.62 call 598
EVENT: 1513179374.63 call 599
EVENT: 1513179374.64 call 600
EVENT: 1513179374.65 call 601
EVENT: 1513179374.66 call 602
EVENT: 1513179374.67 call 603
EVENT: 1513179374.68 call 604
EVENT: 1513179374.69 call 605
EVENT: 1513179374.7 call 606
EVENT: 1513179374.71 call 607
EVENT: 1513179374.72 call 608
EVENT: 1513179374.73 call 609
EVENT: 151317

EVENT: 1513179377.32 call 868
EVENT: 1513179377.33 call 869
EVENT: 1513179377.34 call 870
EVENT: 1513179377.35 call 871
EVENT: 1513179377.36 call 872
EVENT: 1513179377.37 call 873
EVENT: 1513179377.38 call 874
EVENT: 1513179377.39 call 875
EVENT: 1513179377.4 call 876
EVENT: 1513179377.41 call 877
EVENT: 1513179377.42 call 878
EVENT: 1513179377.43 call 879
EVENT: 1513179377.44 call 880
EVENT: 1513179377.45 call 881
EVENT: 1513179377.46 call 882
EVENT: 1513179377.47 call 883
EVENT: 1513179377.48 call 884
EVENT: 1513179377.49 call 885
EVENT: 1513179377.5 call 886
EVENT: 1513179377.51 call 887
EVENT: 1513179377.52 call 888
EVENT: 1513179377.53 call 889
EVENT: 1513179377.54 call 890
EVENT: 1513179377.55 call 891
EVENT: 1513179377.56 call 892
EVENT: 1513179377.57 call 893
EVENT: 1513179377.58 call 894
EVENT: 1513179377.59 call 895
EVENT: 1513179377.6 call 896
EVENT: 1513179377.61 call 897
EVENT: 1513179377.62 call 898
EVENT: 1513179377.63 call 899
EVENT: 1513179377.64 call 900
EVENT: 151317

# THREADING EXAMPLE

In [6]:
import threading

def call_lambda_with_id(uid, thread_num):
    print 'Thread {} started'.format(thread_num)
    client.calculate_async(uid, months=['01'])
    print 'Thread {} finished'.format(thread_num)
    return

#def worker(num):
#    """thread worker function"""
#    print 'Worker: %s' % num
#    return

def seconds_since(start_time):
    return (datetime.now() - start_time).total_seconds()

requests_per_second = 400

threads = []
start_threads = datetime.now()
thread_num = 1
for uid in uids[:5000]:
    print '{}: Start thread {} for uid {}'.format(seconds_since(start_threads), thread_num, uid)
    t = threading.Thread(target=call_lambda_with_id, args=(uid,thread_num,))
    threads.append(t)
    t.start()
    time.sleep(1.0/requests_per_second)
    #time.sleep(.005)
    thread_num = thread_num + 1

print 'All threads finished in {}'.format(seconds_since(start_threads)) 

0.000239: Start thread 1 for uid 31970436701
Thread 1 started
0.003473: Start thread 2 for uid 1674435601
Thread 2 started
0.006899: Start thread 3 for uid 204451602
Thread 3 started
0.011151: Start thread 4 for uid 3158762201
Thread 4 started
0.015501: Start thread 5 for uid 595989402
Thread 5 started
0.019774: Start thread 6 for uid 894536502
Thread 6 started
0.023467: Start thread 7 for uid 30498125901
Thread 7 started
0.027133: Start thread 8 for uid 3173810801
Thread 8 started
0.030239: Start thread 9 for uid 1789308302
Thread 9 started
0.033537: Start thread 10 for uid 986282601
Thread 10 started
0.0371: Start thread 11 for uid 1784327003
Thread 11 started
0.040548: Start thread 12 for uid 246540301
Thread 12 started
0.046379: Start thread 13 for uid 3154052701
Thread 13 started
0.05006: Start thread 14 for uid 31917048601
Thread 14 started
0.053239: Start thread 15 for uid 617921001
Thread 15 started
0.056039: Start thread 16 for uid 3170818001
Thread 16 started
0.060606: Start 

0.83392: Start thread 110 for uid 2032562701
Thread 110 started
0.837269: Start thread 111 for uid 3174342602
Thread 111 started
0.841743: Start thread 112 for uid 2227281901
Thread 112 started
0.84601: Start thread 113 for uid 2970492902
Thread 113 started
0.851693: Start thread 114 for uid 1863002
Thread 114 started
0.855189: Start thread 115 for uid 2486337401Thread 98 finished

Thread 115 started
0.864147: Start thread 116 for uid 368694302
Thread 116 started
0.867437: Start thread 117 for uid 1365624501
Thread 117 started
Thread 100 finished0.872273: Start thread 118 for uid 26964440301

Thread 118 started
0.876401: Start thread 119 for uid 3143464501
Thread 119 started
Thread 105 finished0.884302: Start thread 120 for uid 2430566202Thread 101 finished


Thread 104 finished
Thread 106 finished
Thread 120 started
Thread 107 finished
Thread 97 finished0.907779: Start thread 121 for uid 3172614901

Thread 121 started
0.911596: Start thread 122 for uid 586941601
Thread 122 started
Thr

Thread 194 finished
Thread 213 started
1.65596: Start thread 214 for uid 1843031902
Thread 214 started
1.661109: Start thread 215 for uid 3169649201
Thread 215 started
1.664544: Start thread 216 for uid 717344702
Thread 216 started
Thread 202 finished
1.671787: Start thread 217 for uid 3139530201
Thread 217 started
Thread 198 finished
1.682624: Start thread 218 for uid 2232963101
Thread 218 started
Thread 201 finished
 Thread 206 finished
1.693341: Start thread 219 for uid 2316593801
Thread 208 finished
1.697005: Start thread 220 for uid 1026770101
Thread 219 started
Thread 220 started
1.702101: Start thread 221 for uid 1611002
Thread 221 started
1.70516: Start thread 222 for uid 1523013001
Thread 222 started
1.708601: Start thread 223 for uid 591619601
Thread 211 finishedThread 223 started

Thread 209 finished1.726201: Start thread 224 for uid 873880301

Thread 212 finished
Thread 224 startedThread 205 finished

1.733777: Start thread 225 for uid 584879802
Thread 225 started
Thread 21

2.474177: Start thread 319 for uid 3136973901
Thread 306 finishedThread 319 started

Thread 307 finished2.483306: Start thread 320 for uid 891234502

Thread 311 finished
 Thread 320 started
Thread 305 finished
 Thread 314 finished
Thread 312 finished
2.502319: Start thread 321 for uid 31796095701Thread 313 finished

Thread 321 started
2.511782: Start thread 322 for uid 2435217801
Thread 322 started
2.516009: Start thread 323 for uid 613423202
Thread 323 started
2.51917: Start thread 324 for uid 3150149401
Thread 324 started
2.52274: Start thread 325 for uid 2294101101
Thread 325 started
2.526367: Start thread 326 for uid 247558201
Thread 326 started
2.529356: Start thread 327 for uid 603159001
Thread 327 started
Thread 309 finished2.536152: Start thread 328 for uid 1170593901

Thread 328 started
2.540455: Start thread 329 for uid 908220901
Thread 329 started
2.54348: Start thread 330 for uid 1305856401
Thread 330 started
2.546928: Start thread 331 for uid 1283297501
Thread 331 started


3.335773: Start thread 431 for uid 1082800301
Thread 431 started
3.338927: Start thread 432 for uid 890641202
Thread 432 started
3.34285: Start thread 433 for uid 2788850101
Thread 433 started
3.346589: Start thread 434 for uid 2852490301
Thread 434 started
3.351957: Start thread 435 for uid 1267338101
Thread 435 started
3.355648: Start thread 436 for uid 603221802
Thread 436 started
Thread 420 finished3.361425: Start thread 437 for uid 27304856201

Thread 437 started
3.365006: Start thread 438 for uid 1198696102
Thread 438 started
Thread 423 finished
 3.372922: Start thread 439 for uid 31812554801
Thread 421 finishedThread 424 finished

Thread 439 started
3.381379: Start thread 440 for uid 29899272001
Thread 440 started
Thread 426 finished
 3.387366: Start thread 441 for uid 31927225801
Thread 422 finished
Thread 441 started
Thread 425 finished
 3.39688: Start thread 442 for uid 1545647001
Thread 442 started
3.401709: Start thread 443 for uid 1262912401
Thread 443 started
Thread 430 f

...

Thread 4572 started
Thread 4566 finished
38.288486: Start thread 4573 for uid 2489353701
Thread 4573 started
38.291666: Start thread 4574 for uid 3154100002
Thread 4574 started
38.29522: Start thread 4575 for uid 1875574601
Thread 4575 started
Thread 4567 finished
38.305987: Start thread 4576 for uid 31916601901
Thread 4569 finished
Thread 4576 started
Thread 4568 finished
38.321092: Start thread 4577 for uid 3130762502
Thread 4577 started
38.325935: Start thread 4578 for uid 247821601
Thread 4578 started
38.328867: Start thread 4579 for uid 27310159802
Thread 4579 started
38.33182: Start thread 4580 for uid 2058782802
Thread 4580 started
38.336577: Start thread 4581 for uid 591213101
Thread 4581 started
38.340703: Start thread 4582 for uid 3158721001
Thread 4582 started
38.345346: Start thread 4583 for uid 3137436902
Thread 4583 started
Thread 4570 finished
38.354056: Start thread 4584 for uid 2231325202
Thread 4584 started
38.359088: Start thread 4585 for uid 3183481902
Thread 4585 s

39.136188: Start thread 4674 for uid 3152941201
Thread 4666 finished
 Thread 4674 started
39.147924: Start thread 4675 for uid 2875461001
Thread 4675 started
39.151475: Start thread 4676 for uid 3158647701
Thread 4676 started
Thread 4669 finished
39.157051: Start thread 4677 for uid 751615001
Thread 4677 started
Thread 4667 finished39.168779: Start thread 4678 for uid 1492898701
Thread 4668 finishedThread 4665 finished

Thread 4678 started

Thread 4670 finished
39.186083: Start thread 4679 for uid 32006062001
Thread 4679 started
39.19186: Start thread 4680 for uid 1314052501
Thread 4680 started
39.194995: Start thread 4681 for uid 2221972002
Thread 4681 started
39.199159: Start thread 4682 for uid 26959684401
Thread 4682 started
39.20262: Start thread 4683 for uid 243880201
Thread 4683 started
39.207122: Start thread 4684 for uid 383949602
Thread 4684 started
39.210433: Start thread 4685 for uid 2102978501
Thread 4685 started
39.214053: Start thread 4686 for uid 1013001
Thread 4686 sta

Thread 4769 finished
Thread 4768 finishedThread 4759 finished
Thread 4764 finished
Thread 4756 finished
Thread 4763 finished

Thread 4770 finished
Thread 4780 started
Thread 4771 finished
Thread 4781 started
40.019274: Start thread 4782 for uid 2887562402
Thread 4782 started
40.022504: Start thread 4783 for uid 291498102
Thread 4783 started
40.025391: Start thread 4784 for uid 31970569301
Thread 4784 started
Thread 4772 finished
40.031973: Start thread 4785 for uid 1299653601
Thread 4775 finished
Thread 4776 finished
 Thread 4785 started
 Thread 4774 finished
40.053725: Start thread 4786 for uid 746820801
Thread 4778 finished
Thread 4786 started
40.062252: Start thread 4787 for uid 2227983201
Thread 4787 started
40.066754: Start thread 4788 for uid 3177847501
Thread 4788 started
40.070086: Start thread 4789 for uid 31915997702
Thread 4789 started
Thread 4779 finished
Thread 4773 finished
40.082034: Start thread 4790 for uid 26960402202
Thread 4777 finished40.089047: Start thread 4791 f

Thread 4859 finished
Thread 4862 finished
Thread 4874 started
40.83773: Start thread 4875 for uid 32003121701
Thread 4875 started
40.841953: Start thread 4876 for uid 15428301
Thread 4876 started
Thread 4869 finished40.846762: Start thread 4877 for uid 1169768601

Thread 4877 started
Thread 4868 finished40.857518: Start thread 4878 for uid 3007176601

 Thread 4878 started
Thread 4870 finished40.866564: Start thread 4879 for uid 1330123801
Thread 4871 finished

Thread 4872 finished
Thread 4879 started
40.880028: Start thread 4880 for uid 26991939501
Thread 4880 started
40.883019: Start thread 4881 for uid 1559953101
Thread 4881 started
40.88778: Start thread 4882 for uid 1399684901
Thread 4882 started
40.891393: Start thread 4883 for uid 29142384601
Thread 4883 started
40.894464: Start thread 4884 for uid 1332357402
Thread 4884 started
40.898506: Start thread 4885 for uid 629655401
Thread 4885 started
40.901831: Start thread 4886 for uid 29897508402
Thread 4886 started
40.906387: Start 

Thread 4965 finished
Thread 4966 finished
Thread 4967 finished41.68302: Start thread 4978 for uid 31913196702

Thread 4978 started
41.686872: Start thread 4979 for uid 608551301
Thread 4979 started
41.691488: Start thread 4980 for uid 983969002
Thread 4980 started
41.699067: Start thread 4981 for uid 2481569902
Thread 4981 started
41.704134: Start thread 4982 for uid 3164894101
Thread 4982 started
41.710183: Start thread 4983 for uid 2038438001
Thread 4983 started
41.714073: Start thread 4984 for uid 3133629901
Thread 4984 started
41.720679: Start thread 4985 for uid 3629723801
Thread 4985 started
Thread 4973 finished
Thread 4971 finished
Thread 4972 finished
41.744011: Start thread 4986 for uid 3168640202
Thread 4969 finished
Thread 4975 finished
Thread 4964 finishedThread 4968 finished
Thread 4986 started
41.765068: Start thread 4987 for uid 2888033501

Thread 4987 started
Thread 4974 finished
Thread 4976 finished41.783301: Start thread 4988 for uid 26959419501

Thread 4988 started
T